# 爬蟲複習

In [72]:
import requests

dir(requests)

['ConnectTimeout',
 'ConnectionError',
 'DependencyWarning',
 'FileModeWarning',
 'HTTPError',
 'NullHandler',
 'PreparedRequest',
 'ReadTimeout',
 'Request',
 'RequestException',
 'RequestsDependencyWarning',
 'Response',
 'Session',
 'Timeout',
 'TooManyRedirects',
 'URLRequired',
 '__author__',
 '__author_email__',
 '__build__',
 '__builtins__',
 '__cached__',
 '__cake__',
 '__copyright__',
 '__description__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__title__',
 '__url__',
 '__version__',
 '_internal_utils',
 'adapters',
 'api',
 'auth',
 'certs',
 'chardet',
 'check_compatibility',
 'codes',
 'compat',
 'cookies',
 'delete',
 'exceptions',
 'get',
 'head',
 'hooks',
 'logging',
 'models',
 'options',
 'packages',
 'patch',
 'post',
 'put',
 'request',
 'session',
 'sessions',
 'status_codes',
 'structures',
 'urllib3',
 'utils',
 'warnings']

In [73]:
# shift-tab #看函數的用法解釋 有docstring
# 看doc string #__doc__ 看物件的說明
# requests.options() 傳一個option的requests, 傳回一個response的物件 <Response [200]> 
# 200為OK, 300為redir 轉址, 400為user , 500 server
requests.options("https://www.ptt.cc/bbs/Gossiping/index.html")



<Response [200]>

In [74]:
import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

resp = requests.get(URL)
resp.text

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>批踢踢實業坊</title>\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-common.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-base.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-custom.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/pushstream.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-print.css" media="print">\n\n\n\n\n\t</head>\n    <body>\n\t\t\n<div class="bbs-screen bbs-content">\n    <div class="over18-notice">\n        <p>本網站已依網站內容分級規定處理</p>\n\n        <p>警告︰您即將進入之看板內容需滿十八歲方可瀏覽。</p>\n\n        <p>若您尚未年滿十八歲，請點選離開。若您已滿十八歲，亦不可將本區之內容派發、傳閱、出售、出租、交給或借予年齡未滿18歲的人士瀏覽，或將本網站內容向該人士出示、播放或放映。</p>\n    </div>\n</div>\n\n<div class="bbs-screen bbs-content cent

## 仔細觀察瀏覽PTT八卦版的流程，並且打開開發者工具

In [75]:
import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

custom_headers = {
    "cookie": "over18=1;"
}

resp = requests.get(URL, headers=custom_headers)
# resp.text
resp.text[1500:1900]
# 成功！

'tn wide disabled">下頁 &rsaquo;</a>\n\t\t\t\t<a class="btn wide" href="/bbs/Gossiping/index.html">最新</a>\n\t\t\t</div>\n\t\t</div>\n\t</div>\n\n\t<div class="r-list-container action-bar-margin bbs-screen">\n\t\t\n\t\t\n            \n        \n        \n\t\t<div class="r-ent">\n\t\t\t<div class="nrec"><span class="hl f2">7</span></div>\n\t\t\t<div class="mark"></div>\n\t\t\t<div class="title">\n\t\t\t\n\t\t\t\t<a href="/bbs/Gossiping/M.1514957382.A.'

## 思考一下PTT的換頁策略

我們需要知道現在最新的頁面是多少

![https://i.imgur.com/MBX0eET.png](https://i.imgur.com/MBX0eET.png)

可以用上次的re模組撰寫regular express來取得

In [76]:
import re

import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

custom_headers = {
    "cookie": "over18=1;"
}

resp = requests.get(URL, headers=custom_headers)

# 這個符號
# ‹-> &lsaquo;
total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1

total_page_cnt

31224

In [77]:
# 整理成function
def get_total_page_cnt():
    URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(URL, headers=custom_headers)

    # 這個符號
    # ‹-> &lsaquo;
    total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1
    return total_page_cnt

## 單頁爬蟲

In [78]:
import re

import requests

def get_list_page(url):
    """爬取PTT列表頁面

    Parameters
    ----------
    url : str
        PTT列表頁UTL，
        如：https://www.ptt.cc/bbs/Gossiping/index31063.html

    Returns
    -------
    list
        頁面上所有PTT內文URL
    """
    HOST = "https://www.ptt.cc"
    
    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(url, headers=custom_headers)
    links = re.findall('<a href="(/bbs/Gossiping/M.+\.html)">.+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links

In [79]:
get_list_page("https://www.ptt.cc/bbs/Gossiping/index31063.html")

['https://www.ptt.cc/bbs/Gossiping/M.1514853038.A.F34.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853127.A.9F6.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853296.A.F08.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853353.A.8FF.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853368.A.FBF.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853479.A.935.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853502.A.C78.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853658.A.805.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853662.A.5A8.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853693.A.C78.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853705.A.6AA.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853820.A.24D.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853881.A.32E.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853900.A.4D2.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853914.A.8C3.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1514853941.A.CE8.html',
 'https://www.ptt.cc/bbs

In [80]:
# import re

# import requests

# def get_list_page(url):
#     HOST = "https://www.ptt.cc"
    
#     custom_headers = {
#         "cookie": "over18=1;"
#     }

#     resp = requests.get(url, headers=custom_headers)
#     links = re.findall('<a href="(/bbs/Gossiping/M.+\.html)">.+</a>', resp.text)
#     for link in links:
#         # Generator
#         # https://wiki.python.org/moin/Generators
#         yield HOST + link

In [81]:
# for link in get_list_page('https://www.ptt.cc/bbs/Gossiping/index31008.html'):
#     print(link)

In [82]:
total_page_cnt = get_total_page_cnt()
total_page_cnt

31224

In [83]:
for page in range(total_page_cnt, total_page_cnt - 10, -1):
    print(page)

31224
31223
31222
31221
31220
31219
31218
31217
31216
31215


In [84]:
import re

import requests

# 一次爬10頁
CRAWL_PAGE_CNT = 10
URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"

total_page_cnt = get_total_page_cnt()

for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
    url = URL_TEMPLATE.format(pg)
    for link in get_list_page(url):
        print(link)

https://www.ptt.cc/bbs/Gossiping/M.1514957382.A.B31.html
https://www.ptt.cc/bbs/Gossiping/M.1514957425.A.28C.html
https://www.ptt.cc/bbs/Gossiping/M.1514957445.A.5B3.html
https://www.ptt.cc/bbs/Gossiping/M.1514957467.A.00F.html
https://www.ptt.cc/bbs/Gossiping/M.1514957509.A.C2D.html
https://www.ptt.cc/bbs/Gossiping/M.1514957570.A.4AE.html
https://www.ptt.cc/bbs/Gossiping/M.1514957610.A.84F.html
https://www.ptt.cc/bbs/Gossiping/M.1514957640.A.501.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossiping/M.1514720863.A.84E.html
https://www.ptt.cc/bbs/Gossiping/M.1514336396.A.B25.html
https://www.ptt.cc/bbs/Gossiping/M.1514738804.A.4A0.html
https://www.ptt.cc/bbs/Gossiping/M.1514956771.A.5D8.html
https://www.ptt.cc/bbs/Gossiping/M.1514956790.A.CFE.html
https://www.ptt.cc/bbs/Gossiping/M.1514956808.A.F18.html
https://www.ptt.cc/bbs/Gossiping/M.1514956812.A.468.html
https://www.ptt.cc/bbs/Gossiping/M.1514956875.A.97A.html
https://www.ptt.cc/bbs/Gossipin

https://www.ptt.cc/bbs/Gossiping/M.1514953379.A.E86.html
https://www.ptt.cc/bbs/Gossiping/M.1514953396.A.9E0.html
https://www.ptt.cc/bbs/Gossiping/M.1514953398.A.822.html
https://www.ptt.cc/bbs/Gossiping/M.1514953407.A.EB2.html
https://www.ptt.cc/bbs/Gossiping/M.1514953426.A.949.html
https://www.ptt.cc/bbs/Gossiping/M.1514953436.A.AE3.html
https://www.ptt.cc/bbs/Gossiping/M.1514953549.A.A24.html
https://www.ptt.cc/bbs/Gossiping/M.1514953552.A.D28.html
https://www.ptt.cc/bbs/Gossiping/M.1514953555.A.39D.html
https://www.ptt.cc/bbs/Gossiping/M.1514953565.A.804.html
https://www.ptt.cc/bbs/Gossiping/M.1514953578.A.D28.html
https://www.ptt.cc/bbs/Gossiping/M.1514953617.A.4BA.html
https://www.ptt.cc/bbs/Gossiping/M.1514953617.A.370.html
https://www.ptt.cc/bbs/Gossiping/M.1514953655.A.E8A.html
https://www.ptt.cc/bbs/Gossiping/M.1514953722.A.77C.html
https://www.ptt.cc/bbs/Gossiping/M.1514953724.A.7E1.html
https://www.ptt.cc/bbs/Gossiping/M.1514953725.A.E71.html
https://www.ptt.cc/bbs/Gossipin

In [85]:
!mkdir /tmp/ptt_data

In [108]:
url = "https://www.ptt.cc/bbs/Gossiping/M.1514855751.A.724.html"

"_".join(url.split('/')[-1].split('.')[:-1]) + '.html'

'M_1514855751_A_724.html'

In [115]:
"_".join(url.split('/')[-1].split('.')[:-1]) 

'M_1514855751_A_724'

In [87]:
import requests

DATA_PATH = '/tmp/ptt_data'

def dump_page(url):
    """爬取PTT列表頁面

    Parameters
    ----------
    url : str
        PTT內文頁URL，
        如：https://www.ptt.cc/bbs/Gossiping/M.1514855751.A.724.html

    Returns
    -------
    str
        檔名filename
    """
    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    
    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(url, headers=custom_headers)
    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

In [88]:
dump_page('https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html')

'M_1514807718_A_FC3.html'

In [89]:
!ls /tmp/ptt_data/

M_1514807718_A_FC3.html


In [90]:
import re

import requests

# 一次爬10頁
CRAWL_PAGE_CNT = 10
URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"

total_page_cnt = get_total_page_cnt()

for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
    url = URL_TEMPLATE.format(pg)
    for link in get_list_page(url):
        print(link)
        dump_page(link)

https://www.ptt.cc/bbs/Gossiping/M.1514957382.A.B31.html
https://www.ptt.cc/bbs/Gossiping/M.1514957425.A.28C.html
https://www.ptt.cc/bbs/Gossiping/M.1514957445.A.5B3.html
https://www.ptt.cc/bbs/Gossiping/M.1514957467.A.00F.html
https://www.ptt.cc/bbs/Gossiping/M.1514957509.A.C2D.html
https://www.ptt.cc/bbs/Gossiping/M.1514957570.A.4AE.html
https://www.ptt.cc/bbs/Gossiping/M.1514957610.A.84F.html
https://www.ptt.cc/bbs/Gossiping/M.1514957640.A.501.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossiping/M.1514720863.A.84E.html
https://www.ptt.cc/bbs/Gossiping/M.1514336396.A.B25.html
https://www.ptt.cc/bbs/Gossiping/M.1514738804.A.4A0.html
https://www.ptt.cc/bbs/Gossiping/M.1514956771.A.5D8.html
https://www.ptt.cc/bbs/Gossiping/M.1514956790.A.CFE.html


KeyboardInterrupt: 

# 以上為上次複習

# 再來我們開看看怎麼將HTML的內文整理成稍微結構化的資料格式

In [ ]:
page = {
    "title": "[問卦] 大尾鱸鰻 vs 與神同行？",
    "content": "兩部我都看完了，覺得差不多好看！",
    "author": "chiafann (NAZZZ)",
    "pub_date": "Mon Jan  1 19:55:15 2018",
    "board": "Gossiping",
    "ip": "185.89.251.21"
}

In [116]:
!pip install beautifulsoup4 lxml

In [122]:
from bs4 import BeautifulSoup

html = """
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width" />
    <title></title>

  </head>
  <body>
    <h1>Hello world</h1>

    <p class="welcome">welcome to my python etl website</p>
    <p class="author">Ian Chen</p>
    <p class="contact">ianchen06@gmail.com</p>
    
    <p class="important">Python is cool</p>
    
    <ul>
    <li class="important">Javascript is cool</p>
    </ul>

    <div class="container">
      <ul id="syllabus">
        <li class="odd important">Python</li>
        <li class="even">爬蟲</li>
        <li class="odd important">爬蟲2</li>
        <li class="even important">資料庫</li>
        <li class="odd">高性能爬蟲</li>
      </ul>
    </div>
  </body>
</html>
"""

soup = BeautifulSoup(html, 'lxml') # Beautifulsoup Object
type(soup)

bs4.BeautifulSoup

In [120]:
author = soup.select_one('p.author') #bs4.element.Tag #select_one傳回Tag物件
author

<p class="welcome">welcome to my python etl website</p>

In [94]:
author.text

'Ian Chen'

In [95]:
soup.select('p.author')[0]

<p class="author">Ian Chen</p>

In [96]:
soup.select('div.container')[0].select('ul')[0].select('li.important')

[<li class="odd important">Python</li>,
 <li class="odd important">爬蟲2</li>,
 <li class="even important">資料庫</li>]

In [97]:
soup.select('li.important')

[<li class="important">Javascript is cool
     </li>,
 <li class="odd important">Python</li>,
 <li class="odd important">爬蟲2</li>,
 <li class="even important">資料庫</li>]

In [98]:
soup.select('#syllabus > li.important')

[<li class="odd important">Python</li>,
 <li class="odd important">爬蟲2</li>,
 <li class="even important">資料庫</li>]

# 實際讀取一個ptt的html檔案進來

In [128]:
from bs4 import BeautifulSoup

with open('/tmp/ptt_data/M_1514336396_A_B25.html') as f:
    html = f.read()

soup = BeautifulSoup(html, 'lxml')

In [100]:
type(soup)

bs4.BeautifulSoup

In [129]:
soup.select('.article-meta-value')

[<span class="article-meta-value">Kay731 (K少)</span>,
 <span class="article-meta-value">Gossiping</span>,
 <span class="article-meta-value">Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊</span>,
 <span class="article-meta-value">Wed Dec 27 08:59:55 2017</span>]

In [130]:
metas = [tag.text for tag in soup.select('.article-meta-value')]
metas

['Kay731 (K少)',
 'Gossiping',
 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊',
 'Wed Dec 27 08:59:55 2017']

In [103]:
page = {}
page['author']   = metas[0]
page['board']    = metas[1]
page['title']    = metas[2]
page['pub_date'] = metas[3]

In [131]:
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [124]:
dict_key = ['author', 'board', 'title', 'pub_date']
meta_list = ['Kay731 (K少)', 'Gossiping', 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊', 'Wed Dec 27 08:59:55 2017']

for k, v in zip(dict_key, meta_list):
    print("%s : %s"%(k,v))

author : Kay731 (K少)
board : Gossiping
title : Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊
pub_date : Wed Dec 27 08:59:55 2017


In [125]:
dict_key = ['author', 'board', 'title', 'pub_date']
meta_list = ['Kay731 (K少)', 'Gossiping', 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊', 'Wed Dec 27 08:59:55 2017']

page = {}
for k, v in zip(dict_key, meta_list):
    #print("%s : %s"%(k,v))
    page[k] = v
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [132]:
meta_list = [tag.text for tag in soup.select('span.article-meta-value')]
meta_list

['Kay731 (K少)',
 'Gossiping',
 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊',
 'Wed Dec 27 08:59:55 2017']

In [133]:
dict_key = ['author', 'board', 'title', 'pub_date']
# meta_list = ['Kay731 (K少)', 'Gossiping', 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊', 'Wed Dec 27 08:59:55 2017']

# dictionary comprehension
page = {k:v for k, v in zip(dict_key, meta_list)}
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [134]:
import re

page['ip'] = re.findall('來自: (\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', html)[0]
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

# 內文比較複雜

因為內文沒有被tag包住，可以直接select

所以只能弄刪去法, 把不是內文的東西刪掉

In [144]:
soup.select_one('#main-content').text[:100]

'\n\n作者: nswitch (Nswitch) 看板: BigBanciao\n標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊\n時間: Wed Dec 27 08:42:37 2017\n\n\n'

In [145]:
for line in soup.select('.article-metaline'):
    # extract就像list/dict的del，不會回傳東西，只會modify Object
    line.extract()

In [146]:
for line in soup.select('.article-metaline-right'):
    # extract就像list/dict的del，不會回傳東西，只會modify Object
    line.extract()

In [140]:
soup.select('.article-metaline')

[]

In [141]:
soup.select('.article-metaline-right')

[]

In [147]:
soup.select('#main-content')[0].text[:100]

'\n\n作者: nswitch (Nswitch) 看板: BigBanciao\n標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊\n時間: Wed Dec 27 08:42:37 2017\n\n\n'

In [154]:
content = soup.select_one('div#main-content')

selectors_to_del = ['.article-metaline',
                   '.article-metaline-right',
                    'span.f2',
                    'div.push'
                   ]

for selector in selectors_to_del:
    [tag.extract() for tag in content.select(selector)]
content

<div class="bbs-screen bbs-content" id="main-content">

作者: nswitch (Nswitch) 看板: BigBanciao
標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊
時間: Wed Dec 27 08:42:37 2017


因朋友權限不足 是以代po
並請友版版主轉文章並置底
非常感謝您 真的很感謝

內容
請廣大的鄉民們幫忙或可轉至其他社群幫忙尋找

時間: 12/24 星期日12:40～14:00這段時間遺失

地點:新北市耶誕城市民廣場處

因為家人那天帶小孩去新北耶誕城廣場玩
在玩了旋轉木馬和飛椅之後
發現脖子上的鑽石銀項鍊遺失了
可能是鍊子斷掉了
（特色是鑽石會自然晃動）
後來有請大會廣播並再找了一天都沒找到
甚至從縫隙看水溝蓋裡
也至警局備案 但目前仍沒有找到

鑽石重量十分不到 一般市價幾千元
但那鑽石是由家人做成的生命寶石
對我們意義重大 我們遺失非常的難過
所以 真的能希望由廣大版友幫忙 能夠找回
若有在那裡有撿到或是您的親友有看到拾得的話
請Line: Nswitch
或可交至耶誕城大會服務台
也可送附近警察局再轉交給海山分局

若是可能
會用超放大鏡去看腰間刻的英文名和寶石編號
經確定是我們遺失的
我們會給您台幣一百萬元感謝您有好好地照顧與保護
並在新北廣場發1000份雞排感謝ptt fb 或其他社群的朋友/恩人

附上同款項鍊圖片和新北耶誕城遺失位置圖片連結
<a href="https://imgur.com/gallery/KPuIH" rel="nofollow" target="_blank">https://imgur.com/gallery/KPuIH</a>

--

</div>

In [150]:
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [151]:
page['content'] = content.text.strip()

In [152]:
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'content': '作者: nswitch (Nswitch) 看板: BigBanciao\n標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊\n時間: Wed Dec 27 08:42:37 2017\n\n\n因朋友權限不足 是以代po\n並請友版版主轉文章並置底\n非常感謝您 真的很感謝\n\n內容\n請廣大的鄉民們幫忙或可轉至其他社群幫忙尋找\n\n時間: 12/24 星期日12:40～14:00這段時間遺失\n\n地點:新北市耶誕城市民廣場處\n\n因為家人那天帶小孩去新北耶誕城廣場玩\n在玩了旋轉木馬和飛椅之後\n發現脖子上的鑽石銀項鍊遺失了\n可能是鍊子斷掉了\n（特色是鑽石會自然晃動）\n後來有請大會廣播並再找了一天都沒找到\n甚至從縫隙看水溝蓋裡\n也至警局備案 但目前仍沒有找到\n\n鑽石重量十分不到 一般市價幾千元\n但那鑽石是由家人做成的生命寶石\n對我們意義重大 我們遺失非常的難過\n所以 真的能希望由廣大版友幫忙 能夠找回\n若有在那裡有撿到或是您的親友有看到拾得的話\n請Line: Nswitch\n或可交至耶誕城大會服務台\n也可送附近警察局再轉交給海山分局\n\n若是可能\n會用超放大鏡去看腰間刻的英文名和寶石編號\n經確定是我們遺失的\n我們會給您台幣一百萬元感謝您有好好地照顧與保護\n並在新北廣場發1000份雞排感謝ptt fb 或其他社群的朋友/恩人\n\n附上同款項鍊圖片和新北耶誕城遺失位置圖片連結\nhttps://imgur.com/gallery/KPuIH\n\n--',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

# Youbike

In [155]:
import requests

URL = 'http://data.taipei/youbike'

resp = requests.get(URL)
resp.text[:100]

'{"retCode":1,"retVal":{"0001":{"sno":"0001","sna":"捷運市政府站(3號出口)","tot":"180","sbi":"119","sarea":"信義'

In [156]:
import json

In [162]:
youbike_dict = json.loads(resp.text)

In [158]:
youbike_dict.keys()

dict_keys(['retCode', 'retVal'])

In [ ]:
youbike_dict['retVal']['0001']

In [ ]:
import requests

URL = 'http://data.taipei/youbike'

resp = requests.get(URL)
#resp.json()

# Mobile01

In [169]:
!mkdir /tmp/mobile01_data

mkdir: cannot create directory ‘/tmp/mobile01_data’: File exists


In [193]:
import re
import time

import requests

DATA_PATH = '/tmp/mobile01_data'
CRAWL_PAGE_CNT = 10
#URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"
URL_TEMPLATE = "https://www.mobile01.com/topiclist.php?f=383&p={}"
HOST = "https://www.mobile01.com"


def get_w_cookie(url):
    """GET HTTP url with custom header containing cookie info for PTT

    Parameters
    ----------
    url : str
        PTT八卦版需要驗證年齡的URL，
        如：https://www.ptt.cc/bbs/Gossiping/index31063.html

    Returns
    -------
    Response
        Requestsµ模組的Response Object
    """
    custom_headers = {
        "cookie": "over18=1;"
    }
    resp = requests.get(url, headers=custom_headers)
    return resp


def get_total_page_cnt():
    """取得現在ptt板塊的總頁數

    Parameters
    ----------

    Returns
    -------
    int
       現在ptt板塊的總頁數
    """
    url = URL_TEMPLATE.format('')
    resp = get_w_cookie(url)

    # 這個符號
    # ‹-> &lsaquo;
    total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1
    return total_page_cnt

def get_list_page(url):
    """GET列表頁，取得內文頁的連結們

    Parameters
    ----------
    url : str
        PTT 列表頁URL

    Returns
    -------
    list
       PTT內文頁的links
    """
    resp = get_w_cookie(url)
    links = re.findall('<a href="(topicdetail.php\?f=383&t=\d+)" class="topic_gen" title="人氣: \d+">', resp.text)
    detail_page_links = ["%s/%s"%(HOST, link) for link in links]
    return detail_page_links

def dump_page(url):
    """GET url的HTML並且寫到檔案裡

    Parameters
    ----------
    url : str
        PTT 內文頁URL

    Returns
    -------
    str
       儲存的檔案名稱
    """
    filename = "_".join(re.findall('f=(\d+)&t=(\d+)&p=(\d+)', url)[0]) + '.html'
    resp = get_w_cookie(url)

    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return resp

In [210]:
for pg in range(1,2):
    url = URL_TEMPLATE.format(pg)
    for link in get_list_page(url):
        link_pg = 1
        while True:
            """
            內文頁換頁邏輯
            """
            link_url = link + "&p=%s"%link_pg
            print(link_url)
            resp = dump_page(link_url)
            time.sleep(0.4)
            
            """
            問老師
            用canonical address來做判斷是否爬到最後一頁
            """
            canonical = re.findall('<link rel="canonical" href="(.+)" />', resp.text)[0]
            if link_pg == 1: #第一頁的特別處理，拿掉p=1
                link_url = link_url[:-4]
            if canonical != link_url:
                break
            link_pg += 1

https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=1
https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=2
https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=3
https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=4
https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=5
https://www.mobile01.com/topicdetail.php?f=383&t=5357785&p=6


KeyboardInterrupt: 

# Facebook GraphAPI

In [1]:
import requests

ACCESS_TOKEN = "" # developers.facebook.com -> Tools & Support -> Get token

resp = requests.get("https://graph.facebook.com/v2.11/1068531466501015?fields=feed%7Bcomments%2Cmessage%7D&access_token=%s"%(ACCESS_TOKEN))

ValueError: unsupported format character 'B' (0x42) at index 63

In [ ]:
resp.json()

# 瀏覽器自動化

chromedriver

https://chromedriver.storage.googleapis.com/index.html?path=2.34/

打開Terminal
```bash
cp Downloads/chromedriver /usr/local/bin/
```

In [ ]:
!pip install splinter

In [ ]:
import splinter

b = splinter.Browser('chrome')

In [ ]:
b.visit("https://airbnb.com")

In [ ]:
button = b.find_by_xpath('//*[@id="site-content"]/div[1]/div/div/div/div[2]/div/div/div/div/button')[0]

In [ ]:
button.click()

In [ ]:
b.fill('location', 'Paris\n')

In [ ]:
homes = b.find_by_xpath('//*[@id="site-content"]/div/div/div[2]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/div/button/div/div/div[2]/div/div/span')[0]

In [ ]:
homes.click()

In [ ]:
b.execute_script("console.log('hi bb105')")

In [ ]:
b.execute_script("window.scrollBy(0,100)")

In [ ]:
b.quit()